In [1]:
import torch
import os
import cv2
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import normalize
import scipy
from PIL import ImageOps
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
from torchvision import transforms
from matplotlib import cm

REBUILD_DATA = True

In [2]:
training_data = np.load('training_data_alex.npy', allow_pickle=True)

In [3]:
if REBUILD_DATA:
    # Create rotated set
    training_data_rotated = []
    for img in training_data:
        training_data_rotated.append(img)
        for i in range(1,13):
            training_data_rotated.append([scipy.ndimage.rotate(img[0], i, reshape=False), img[1].reshape(1000)])


    # Create flipped set
    training_data_r_f = []
    for img in training_data_rotated:
        training_data_r_f.append(img)
        training_data_r_f.append([np.fliplr(img[0]), img[1]])

#     remove un-needed noise
    def dumb_remover(img):
        better_img = np.ndarray((224, 224, 3))
        for row in range(len(img)):
            for col in range(len(img[row])):
                better_img[row][col] = img[row][col]
                
        return better_img


    for i in tqdm(range(len(training_data_r_f))):
        training_data_r_f[i][0] = dumb_remover(training_data_r_f[i][0]).reshape(224, 224, 3)

    # shuffle for good luck
    np.random.shuffle(training_data_r_f)

    # Save
    np.save('training_data_rotated_and_flipped_alex.npy', training_data_r_f)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5538/5538 [03:05<00:00, 29.85it/s]


In [4]:
training_data = np.load('training_data_rotated_and_flipped_alex.npy', allow_pickle=True)

In [5]:
X = torch.Tensor([i[0] for i in training_data]).view(-1, 224, 224, 3)
X = X/255.0
print("X shape: ", X.shape)

y = torch.Tensor([i[1] for i in training_data])
print("y shape: ", y.shape)

VAL_PCT = 0.4
val_size = int(len(X)*VAL_PCT)
print(len(X), val_size)

X shape:  torch.Size([5538, 224, 224, 3])
y shape:  torch.Size([5538, 1000])
5538 2215


In [6]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

3323
2215


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
alexnet = models.alexnet(pretrained=False)

In [9]:
alexnet = nn.DataParallel(alexnet)
alexnet.to(device)

DataParallel(
  (module): AlexNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
    (classifier): Sequential(
      (0): Dropout(p=0.5, inplace=False)
   

In [10]:
EPOCHS = 5
BATCH_SIZE = 4

optimizer = optim.Adam(alexnet.parameters(), lr=0.001)
loss_function = nn.MSELoss()

loss_outsample = []
acc = []
loss_insample = []

alex_model_max = alexnet

for epoch in range(EPOCHS):
    alexnet.train()
    for i in range(0, len(train_X), BATCH_SIZE):
        batch_X = train_X[i:i+BATCH_SIZE].view(-1, 3, 224, 224)
        batch_y = train_y[i:i+BATCH_SIZE]
        
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        alexnet.zero_grad()
        outputs = alexnet(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    correct = 0
    total = 0

    max_accuracy = 0
    max_loss = 10
    
    alexnet.eval()
    with torch.no_grad():
        for i in range(len(test_X)):
            real_class = torch.argmax(test_y[i]).to(device)
            model_out = alexnet(test_X[i].view(-1, 3, 224, 224).to(device))[0]
            predicted_class = torch.argmax(model_out)
#             print(predicted_class, real_class)
            loss_test = loss_function(model_out, test_y[i].to(device))
            if predicted_class == real_class:
                correct += 1
            total += 1
    acc.append(round(correct/total, 4))
    print("Out/Sample Accuracy: ", round(correct/total, 3))
    loss_outsample.append(loss_test)
    loss_insample.append(loss)
    print("Loss: ", loss)
    if round(correct/total, 3) > max_accuracy:
        max_accuracy = round(correct/total, 3)
        max_loss = loss
        alex_model_max = alexnet
    elif (round(correct/total, 3) == max_accuracy) and (loss < max_loss):
        max_loss = loss
        alex_model_max = alexnet
    

c:\users\baby yoda\pycharmprojects\dltutorial_2\venv\lib\site-packages\torch\cuda\nccl.py:24: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn('PyTorch is not compiled with NCCL support')


Out/Sample Accuracy:  0.195
Loss:  tensor(0.0003, device='cuda:0', grad_fn=<MseLossBackward>)
Out/Sample Accuracy:  0.195
Loss:  tensor(0.0003, device='cuda:0', grad_fn=<MseLossBackward>)
Out/Sample Accuracy:  0.195
Loss:  tensor(0.0003, device='cuda:0', grad_fn=<MseLossBackward>)
Out/Sample Accuracy:  0.195
Loss:  tensor(0.0003, device='cuda:0', grad_fn=<MseLossBackward>)
Out/Sample Accuracy:  0.195
Loss:  tensor(0.0003, device='cuda:0', grad_fn=<MseLossBackward>)
